# The dojo

## Setup training device

In [31]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [32]:
import torch
from torch import device, nn
import torchvision


print(f'PyTorch version: {torch.__version__}\ntorchvision version: {torchvision.__version__}')

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'Using device: {device}')

PyTorch version: 1.13.1
torchvision version: 0.14.1
Using device: cpu


## Data loading

In [33]:
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset
from torch.utils.data import Dataset, DataLoader

MNIST_train_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
    target_transform=None
)

MNIST_test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_dir = img_dir
        self.img_names = pd.read_csv(img_dir + "/../../" + annotations_file)        
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_names)

    def __getitem__(self, idx):        
        img_path = os.path.join(self.img_dir, self.img_names.iloc[idx, 0])
        image = read_image(img_path)        
        if self.transform:
            image = self.transform(image)
        return image.float()


MNIST_class_names = MNIST_train_data.classes

train_data = CustomImageDataset("indices/trainIndex/bremen.csv","data/leftImg8bit/train/bremen/")
test_data = CustomImageDataset("indices/testIndex/berlin.csv","data/leftImg8bit/test/berlin/")

# Turn data into batches (mini-batches)
# More computationally efficent, so we dont store all data in memory
# Gives neural network more changes to update its gradients per epoch
# andrew ng minibatches

BATCH_SIZE = 5

train_dataloader = DataLoader(train_data,
                              batch_size=BATCH_SIZE,
                              shuffle=True)

test_dataloader = DataLoader(test_data,
                             batch_size=BATCH_SIZE,
                             shuffle=False)

MNIST_train_dataloader = DataLoader(MNIST_train_data,
                              batch_size=BATCH_SIZE,
                              shuffle=True)

MNIST_test_dataloader = DataLoader(MNIST_test_data,
                             batch_size=BATCH_SIZE,
                             shuffle=False)

print(f'Len of train dataloader: {len(train_dataloader)} batches of {BATCH_SIZE}')
print(f'Len of test dataloader: {len(test_dataloader)} batches of {BATCH_SIZE}')

Len of train dataloader: 63 batches of 5
Len of test dataloader: 109 batches of 5


### Check out what's inside the training dataloader

In [34]:
train_features_batch = next(iter(train_dataloader))
MNIST_train_features_batch, MNIST_train_labels_batch = next(iter(MNIST_train_dataloader))

## Model

### Create a flatten layer - Testing Flatten()

In [35]:
flatten_model = nn.Flatten()

x = train_features_batch[0]
print(f'Shape before flattening: {x.shape}')     # torch.Size([3, 1024, 2048])

output = flatten_model(x)
print(f'Shape after flattening: {output.shape}') # torch.Size([3, 2097152])

Shape before flattening: torch.Size([3, 1024, 2048])
Shape after flattening: torch.Size([3, 2097152])


### Instantiate model

In [36]:
from model.chrome_vision import ChromeVisionModel

model = ChromeVisionModel(
    input_shape= 1024 * 2048,    
    hidden_units=5,    # Units in the hidden layer
    output_shape= 3 # one for every class (Arbitrary right now)
).to(device)

In [37]:
from model.chrome_vision import ChromeVisionModelV2

model_MNIST_data = ChromeVisionModel(
    input_shape=1,      # Color channel: black/white 
    hidden_units=10,    # Units in the hidden layer
    output_shape=(len(MNIST_class_names)) # Each class
).to(device)

### Setup loss function and optimizer

In [38]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.1)

### Training loop

In [39]:
from timeit import default_timer as timer
from tqdm.auto import tqdm
from model.evaluation import train_step, test_step, accuracy_fn # use torchmetrics.Accuracy()
from model.utilis import print_train_time

torch.manual_seed(42)
train_time_start_on_cpu = timer()

epochs = 3
for epoch in tqdm(range(epochs)):
    print(f'Epoch: {epoch}\n')

    train_step(model=model,
               data_loader=train_dataloader,
               loss_fn=loss_fn,
               optimizer=optimizer,
               accuracy_fn=accuracy_fn,
               device=device)
    
    # Cant currently test on our own data
    # test_step(model=model,
    #            data_loader=test_dataloader,
    #            loss_fn=loss_fn,
    #            accuracy_fn=accuracy_fn,
    #            device=device)

# Print time taken
train_time_end_on_cpu = timer()
total_train_time_model = print_train_time(train_time_start_on_cpu, train_time_end_on_cpu, str(next(model.parameters()).device))

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch: 0



 33%|███▎      | 1/3 [00:40<01:21, 40.68s/it]

Train loss: 0.00000 | Train acc: 0.00%
Epoch: 1



 67%|██████▋   | 2/3 [01:20<00:40, 40.40s/it]

Train loss: 0.00000 | Train acc: 0.00%
Epoch: 2



100%|██████████| 3/3 [01:56<00:00, 38.74s/it]

Train loss: 0.00000 | Train acc: 0.00%
Train time on cpu: 116.244 seconds


### Calculate model results on test dataset

In [40]:
# model_results = test_step(model=model,
#                            data_loader=test_dataloader,
#                            loss_fn=loss_fn,
#                            accuracy_fn=accuracy_fn,
#                            device=device)

# model_results